In [2]:
import os

In [2]:
%pwd

'd:\\end-to-end-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Bappy\\YouTube\\Kidney-Disease-Classification-Deep-Learning-Project'

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
import dagshub
dagshub.init(repo_owner='keerthikapopuri', repo_name='end-to-end-classification', mlflow=True)

Accessing as keerthikapopuri

Initialized MLflow to track repo "keerthikapopuri/end-to-end-classification"

Repository keerthikapopuri/end-to-end-classification initialized!

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = Path(r"D:\end-to-end-classification\config\config.yaml"),
        params_filepath = Path(r"D:\end-to-end-classification\params.yaml")):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/keerthikapopuri/end-to-end-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.1
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="MobilenetModel")
            else:
                mlflow.keras.log_model(self.model, "model")


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 47 images belonging to 2 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 364ms/step - accuracy: 0.1596 - loss: 321.7097  


2024/09/25 12:56:42 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/25 12:56:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'MobilenetModel' already exists. Creating a new version of this model...
2024/09/25 12:57:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MobilenetModel, version 2
Created version '2' of model 'MobilenetModel'.
2024/09/25 12:57:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-hen-52 at: https://dagshub.com/keerthikapopuri/end-to-end-classification.mlflow/#/experiments/0/runs/d39e50d552914bc3b33ce00fb9f483d6.
2024/09/25 12:57:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/keerthikapopuri/end-to-end-classification.mlflow/#/e